In [24]:
# from scripts.predict import run_prediction
import json
import fitz
import os
import re
import time
from datetime import datetime as dt
import ntpath
import csv

In [25]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    squad_convert_examples_to_features
)
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits

In [26]:
# import gradio as gr

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# device = 'cuda'
# check = torch.cuda.current_device()
# print(torch.cuda.get_device_name(check))
torch.cuda.is_available()

cuda:0


True

In [28]:
def path_leaf(path):
    if path:
        head, tail = ntpath.split(path)
        return head, tail or ntpath.basename(head)
    else:
        return None, None

In [29]:
def pdf_toTxt0(from_path):
    files = os.listdir(from_path)
    contracts_text = []
    for file in files:
        if file.endswith(".pdf"):
            print("processing :", file)
            fpath, fname = path_leaf(file)
            with fitz.open(os.path.join(from_path, file)) as doc:
                text = ""
                for page in doc:
                    txt = page.getText()
                    # cleaned = re.sub(r'\s+', ' ', txt)
                    text += txt
            with open(os.path.join(to_path, fname+".txt"), "w", encoding="utf-8") as f:
                f.write(text)
    print('All files converted to text!')
    
    
def pdf_toTxt(from_path, file):
    with fitz.open(os.path.join(from_path, file)) as doc:
        text = ""
        for page in doc:
            txt = page.getText()
            # cleaned = re.sub(r'\s+', ' ', txt)
            text += txt
    return text

In [30]:
with open('training/cuad-data/CUADv1.json') as json_file:
    data = json.load(json_file)

In [31]:
# questions = []
# for i, q in enumerate(data['data'][0]['paragraphs'][0]['qas']):
#     question = data['data'][0]['paragraphs'][0]['qas'][i]['question']
#     questions.append(question)
# contract = data['data'][0]['paragraphs'][0]['context']

In [32]:
def read_contents(filepath):
    with open(filepath, 'r', encoding="utf-8") as file:
        data = file.read() #.replace('\n', ' ')
    return data

### Load Model

In [33]:
%%time
# def load_mdl(model_path):
model_path = 'models/roberta-base/'
config_class, model_class, tokenizer_class = (
    AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer)
config = config_class.from_pretrained(model_path)
tokenizer = tokenizer_class.from_pretrained(
    model_path, do_lower_case=True, use_fast=False)
model = model_class.from_pretrained(model_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

processor = SquadV2Processor()
print('Model Loaded!')
# return tokenizer, model, processor

Model Loaded!
Wall time: 2.29 s


In [34]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [35]:
def get_examples(question_texts, context_text):
    examples = []
    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            answers=None,)
        examples.append(example)
        
    return examples

In [36]:
def get_allresults(eval_dataloader, features):
    
    all_results = []

    for batch in eval_dataloader:
        
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs.to_tuple()]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)
                
    return all_results

In [37]:
def run_prediction(question_texts, context_text):
    
    ### Setting hyperparameters
    max_seq_length = 512
    doc_stride = 256
    n_best_size = 1
    max_query_length = 64
    max_answer_length = 512
    do_lower_case = False
    null_score_diff_threshold = 0.0

    # model_name_or_path = "../models/roberta-base/"
    stt = dt.now()
    print(stt)
    print('Generating Examples..')
    examples = get_examples(question_texts, context_text)
    print('Examples Generated..', dt.now()-stt)

    print('Converting Examples to Features..')
    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )
    print('Examples Converted to Features..', dt.now()-stt)

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    print('Generating all results..') # Model required at this step
    all_results = get_allresults(eval_dataloader, features)
    print('All results Generated..', dt.now()-stt)

    print('Getting final predictions..')
    final_predictions = compute_predictions_logits(
        all_examples=examples,
        all_features=features,
        all_results=all_results,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        do_lower_case=do_lower_case,
        output_prediction_file=None,
        output_nbest_file=None,
        output_null_log_odds_file=None,
        verbose_logging=False,
        version_2_with_negative=True,
        null_score_diff_threshold=null_score_diff_threshold,
        tokenizer=tokenizer
    )
    print('Got final predictions..', dt.now()-stt)
    return final_predictions

In [38]:
# 2022-08-29 22:52:04.427310
# Generating Examples..
# Examples Generated.. 0:00:02.034990
# Converting Examples to Features..
# convert squad examples to features: 100%|██████████████████████████████████████████████| 18/18 [10:11<00:00, 33.95s/it]
# add example index and unique id: 100%|███████████████████████████████████████████████| 18/18 [00:00<00:00, 8651.02it/s]
# Examples Converted to Features.. 0:10:18.720789
# Generating all results..
# All results Generated.. 0:17:39.483510
# Getting final predictions..
# Got final predictions.. 0:17:40.406732
# Wall time: 17min 40s

In [39]:
# %%time 
# predictions = run_prediction(questions, contract)

In [40]:
# %%time
# with open('output/predictions.txt', 'w') as f:
#     for i, p in enumerate(predictions):
#         f.write(f"Question {i+1}: {questions[int(p)]}\nAnswer: {predictions[p]}\n\n")

In [41]:
%%time
def parse_contract(contract, to_path, file, questionsdict):
    lencon = len(contract.split())
    print('Length of Contract ', lencon)
    if lencon > 30000:
        for part in range(0, lencon, 30000):
            partcontract = contract[part:part+30000]
            predictions = run_prediction(questions, partcontract)

            with open(to_path+'predictions_{}_{}_{}.txt'.format(file, str(part),str(part+30000)), 'w') as f:
                for i, p in enumerate(predictions):
                    f.write(f"Question {i+1}: {questions[int(p)]}\nAnswer: {predictions[p]}\n\n")
                    
            with open(to_path+'predictions_{}_{}_{}.csv'.format(file, str(part),str(part+30000)), 'a') as f:
                writer=csv.writer(f, delimiter='\t',lineterminator='\n',)
                row, header = [file], []
                for i, p in enumerate(predictions):
                    header.append(questionsdict[i])
                    row.append(predictions[p])
                    writer.writerow(header)
                    writer.writerow(row)

    else:
        predictions = run_prediction(questions, contract)

        with open(to_path+'predictions_{}.txt'.format(file), 'w') as f:
            for i, p in enumerate(predictions):
                f.write(f"Question {i+1}: {questions[int(p)]}\nAnswer: {predictions[p]}\n\n")
                
    print('All predictions completed!')

Wall time: 0 ns


In [42]:
questionsdict = {
    0:"File Name",
    1:"Document Name",
    2:"Parties",
    3:"Agreement Date",
    4:"Effective Date",
    5:"Expiration Date",
    6:"Renewal Term",
    7:"Notice Period To Terminate Renewal",
    8:"Termination For Convenience",
    9:"Revenue/Profit Sharing",
    10:"Price Restrictions",
    11:"Minimum Commitment",
    12:"Volume Restriction",
    13:"Irrevocable Or Perpetual License",
    14:"Post-Termination Services",
    15:"Audit Rights",
    16:"Uncapped Liability",
    17:"Cap On Liability",
    18:"Insurance"
}

In [43]:
%%time

from_path = "D:\\Projects\\legalcontracts\\demo\\input\\contracts\\"
to_path = "D:\\Projects\\legalcontracts\\demo\\output\\"
qfile = "D:\\Projects\\legalcontracts\\demo\\input\\Mapped_Fields.txt"
questions = open(qfile, 'r').read().split('|')

files = os.listdir(from_path)
for file in files:
    if file.lower().endswith('.pdf'):
        contract = pdf_toTxt(from_path, file)
        # contract = read_contents(cfile)
        # print(len(contract.split()))
        parse_contract(contract, to_path, file, questionsdict)

Length of Contract  2435
2022-09-15 08:40:35.133946
Generating Examples..
Examples Generated.. 0:00:00.142623
Converting Examples to Features..


add example index and unique id: 100%|██████████████████████████████████████████████| 18/18 [00:00<00:00, 17383.71it/s]


Examples Converted to Features.. 0:00:11.600558
Generating all results..
All results Generated.. 0:00:34.428014
Getting final predictions..
Got final predictions.. 0:00:34.481870
All predictions completed!
Length of Contract  1305
2022-09-15 08:41:09.644180
Generating Examples..
Examples Generated.. 0:00:00.078792
Converting Examples to Features..


add example index and unique id: 100%|█████████████████████████████████████████████████████████| 18/18 [00:00<?, ?it/s]


Examples Converted to Features.. 0:00:06.788682
Generating all results..
All results Generated.. 0:00:18.366303
Getting final predictions..
Got final predictions.. 0:00:18.398219
All predictions completed!
Wall time: 52.9 s
